In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


### Collecting results for alleles

Setup for script

In [2]:
alleles = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCIIpan_alleles.txt').readlines() if x.strip()[:2] != 'H-']
len(alleles)

5620

In [3]:
def collect_allele_BR(category):
    mutations = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/residues/{0}.txt'.format(category)).readlines()]
    df = pd.DataFrame({'mutation': mutations})
    
    for i, allele in enumerate(alleles):
        if i % 100 == 0:
            print i
        BR = []
        aff = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/affinities/{0}/{1}.csv'.format(category, allele), sep='\t', skiprows=1)
        for mutation in mutations: 
            BR.append(aff[aff.ID == mutation].Rank.min())
        df[allele] = BR
    df.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/allele_matrices/{0}.csv'.format(category))

Running on the cluster

In [5]:
def create_cluster_script_alleles(categories):
    
    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/allele_matrices.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(categories)))
        #out_file.write("#$ -t 1-2\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=2G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set categories=({0})\n".format(" ".join(categories)))
        out_file.write("\n")

        out_file.write("set category=$categories[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
  
        out_file.write("python /cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/creating_allele_matrices.py $category\n")
        out_file.write("\n")
                       
        out_file.write("date\n")

In [6]:
categories = ['passenger']
create_cluster_script_alleles(categories)

### Merging results into patients

Setup for script

In [4]:
def PHBR(x):
    number_of_alleles = len(x)
    s = 0
    for i in range(number_of_alleles):
        s += 1/float(x[i])
    return number_of_alleles / float(s)

In [5]:
def create_patient_matrix(category):
    df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/allele_matrices/{0}.csv'.format(category), index_col=0)
    patients_used = []
    for patient in patient_dictionary.keys():
        patient_alleles = patient_dictionary[patient]
        try:
            df[patient] = df[patient_alleles].apply(PHBR, axis=1)
            patients_used.append(patient)
        except:
            print patient
    df.index = df['mutation']
    df[patients_used].to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/{0}.csv'.format(category))

In [17]:
patient_dictionary = pickle.load(open('/cellar/users/ramarty/Data/hla_ii/hla_types/TCGA.HLA_classII.p'))

In [18]:
patient_dictionary.keys()[:3]

['TCGA-D3-A3CE', 'TCGA-D3-A3CB', 'TCGA-DH-A7UV']

In [19]:
patient_dictionary['TCGA-D3-A3CE']

['DRB1_0101',
 'DRB1_1501',
 'HLA-DPA10103_DPB10401',
 'HLA-DPA10103_DPB10301',
 'HLA-DPA10103_DPB10401',
 'HLA-DPA10103_DPB10301',
 'HLA-DQA10102_DQB10602',
 'HLA-DQA10102_DQB10501',
 'HLA-DQA10101_DQB10602',
 'HLA-DQA10101_DQB10501']

Running on the cluster

In [7]:
def create_cluster_script_patients_DR(categories):
    
    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/patient_matrices.DR.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(categories)))
        out_file.write("#$ -l h_vmem=2G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set categories=({0})\n".format(" ".join(categories)))
        out_file.write("\n")

        out_file.write("set category=$categories[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
  
        out_file.write("python /cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/creating_patient_matrices.DR.py $category\n")
        out_file.write("\n")
                       
        out_file.write("date\n")

In [8]:
categories = ['oncogenes', 'tsgenes']
create_cluster_script_patients_DR(categories)

In [11]:
def create_cluster_script_patients_all(categories):
    
    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/patient_matrices.all.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(categories)))
        out_file.write("#$ -l h_vmem=2G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set categories=({0})\n".format(" ".join(categories)))
        out_file.write("\n")

        out_file.write("set category=$categories[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
  
        out_file.write("python /cellar/users/ramarty/Projects/hla_ii/data_gathering/affinities/creating_patient_matrices.all.py $category\n")
        out_file.write("\n")
                       
        out_file.write("date\n")

In [12]:
categories = ['oncogenes', 'tsgenes']
create_cluster_script_patients_all(categories)

### Merging MHC-I and MHC-II into a single predictor

Debugging

In [13]:
def PHBR(x):
    number_of_alleles = len(x)
    s = 0
    for i in range(number_of_alleles):
        s += 1/float(x[i])
    return number_of_alleles / float(s)

In [14]:
category = 'oncogenes'
patient_dictionary = pickle.load(open('/cellar/users/ramarty/Data/hla_ii/hla_types/TCGA.HLA_classII.p'))

In [15]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/allele_matrices/{0}.csv'.format(category), index_col=0)
patients_used = []
for patient in patient_dictionary.keys()[:10]:
    patient_alleles = patient_dictionary[patient]
    try:
        df[patient] = df[patient_alleles].apply(PHBR, axis=1)
        patients_used.append(patient)
    except:
        print patient
df.index = df['mutation']

TCGA-D3-A3CE
TCGA-D3-A3CB
TCGA-DH-A7UV
TCGA-C5-A1BM
TCGA-C5-A1BL
TCGA-41-3915
TCGA-85-6560
TCGA-2V-A95S
TCGA-VQ-A94R
TCGA-DC-4745


In [18]:
patient_alleles = patient_dictionary[patient]

In [19]:
patient_alleles

['DRB1_0801',
 'DRB1_0801',
 'HLA-DPA10201_DPB10401',
 'HLA-DPA10201_DPB10501',
 'HLA-DPA10103_DPB10401',
 'HLA-DPA10103_DPB10501',
 'HLA-DQA10401_DQB10402',
 'HLA-DQA10401_DQB10402',
 'HLA-DQA10401_DQB10402',
 'HLA-DQA10401_DQB10402']

In [20]:
df[patient_alleles].apply(PHBR, axis=1)

KeyError: "['HLA-DPA10201_DPB10401' 'HLA-DPA10201_DPB10501' 'HLA-DPA10103_DPB10401'\n 'HLA-DPA10103_DPB10501' 'HLA-DQA10401_DQB10402' 'HLA-DQA10401_DQB10402'\n 'HLA-DQA10401_DQB10402' 'HLA-DQA10401_DQB10402'] not in index"

In [22]:
df['HLA-DPA10201_DPB10401']

KeyError: 'HLA-DPA10201_DPB10401'

In [24]:
alleles = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCIIpan_alleles.txt').readlines()]

In [26]:
alleles[4000:]

['HLA-DQA10303-DQB10337',
 'HLA-DQA10303-DQB10338',
 'HLA-DQA10303-DQB10401',
 'HLA-DQA10303-DQB10402',
 'HLA-DQA10303-DQB10403',
 'HLA-DQA10303-DQB10404',
 'HLA-DQA10303-DQB10405',
 'HLA-DQA10303-DQB10406',
 'HLA-DQA10303-DQB10407',
 'HLA-DQA10303-DQB10408',
 'HLA-DQA10303-DQB10501',
 'HLA-DQA10303-DQB10502',
 'HLA-DQA10303-DQB10503',
 'HLA-DQA10303-DQB10505',
 'HLA-DQA10303-DQB10506',
 'HLA-DQA10303-DQB10507',
 'HLA-DQA10303-DQB10508',
 'HLA-DQA10303-DQB10509',
 'HLA-DQA10303-DQB10510',
 'HLA-DQA10303-DQB10511',
 'HLA-DQA10303-DQB10512',
 'HLA-DQA10303-DQB10513',
 'HLA-DQA10303-DQB10514',
 'HLA-DQA10303-DQB10601',
 'HLA-DQA10303-DQB10602',
 'HLA-DQA10303-DQB10603',
 'HLA-DQA10303-DQB10604',
 'HLA-DQA10303-DQB10607',
 'HLA-DQA10303-DQB10608',
 'HLA-DQA10303-DQB10609',
 'HLA-DQA10303-DQB10610',
 'HLA-DQA10303-DQB10611',
 'HLA-DQA10303-DQB10612',
 'HLA-DQA10303-DQB10614',
 'HLA-DQA10303-DQB10615',
 'HLA-DQA10303-DQB10616',
 'HLA-DQA10303-DQB10617',
 'HLA-DQA10303-DQB10618',
 'HLA-DQA103

In [27]:
[x for x in alleles if 'DPA10201' in x]

['HLA-DPA10201-DPB10101',
 'HLA-DPA10201-DPB10201',
 'HLA-DPA10201-DPB10202',
 'HLA-DPA10201-DPB10301',
 'HLA-DPA10201-DPB10401',
 'HLA-DPA10201-DPB10402',
 'HLA-DPA10201-DPB10501',
 'HLA-DPA10201-DPB10601',
 'HLA-DPA10201-DPB10801',
 'HLA-DPA10201-DPB10901',
 'HLA-DPA10201-DPB110001',
 'HLA-DPA10201-DPB11001',
 'HLA-DPA10201-DPB110101',
 'HLA-DPA10201-DPB110201',
 'HLA-DPA10201-DPB110301',
 'HLA-DPA10201-DPB110401',
 'HLA-DPA10201-DPB110501',
 'HLA-DPA10201-DPB110601',
 'HLA-DPA10201-DPB110701',
 'HLA-DPA10201-DPB110801',
 'HLA-DPA10201-DPB110901',
 'HLA-DPA10201-DPB111001',
 'HLA-DPA10201-DPB11101',
 'HLA-DPA10201-DPB111101',
 'HLA-DPA10201-DPB111201',
 'HLA-DPA10201-DPB111301',
 'HLA-DPA10201-DPB111401',
 'HLA-DPA10201-DPB111501',
 'HLA-DPA10201-DPB111601',
 'HLA-DPA10201-DPB111701',
 'HLA-DPA10201-DPB111801',
 'HLA-DPA10201-DPB111901',
 'HLA-DPA10201-DPB112101',
 'HLA-DPA10201-DPB112201',
 'HLA-DPA10201-DPB112301',
 'HLA-DPA10201-DPB112401',
 'HLA-DPA10201-DPB112501',
 'HLA-DPA1020